####Installed generic libraries 

In [0]:
###########################################################################################################
##                 Generic Notebook parameters    - Here we are check workspace enevironment and based on that we are setting the path             
###########################################################################################################
if spark.conf.get("spark.databricks.clusterUsageTags.managedResourceGroup") == 'rg-training-team-dev-01':
    environment = "dev"
    print('This is', environment, "environment")
else:
    environment = "prod"  # Empty means we can consider as Production environment. Created for email delivery
    print('This is', environment, "environment")

# delta_path = getStoragePath(spark.conf.get("spark.databricks.clusterUsageTags.managedResourceGroup"), "silver")
ConfigJobmetadata = f'abfss://config@dlstraining{environment}01.dfs.core.windows.net/config/job_metadata_details/delta'
print(ConfigJobmetadata)

Generate Personal access token and Access from keyvault for databriks API connectivity

In [0]:
instance_name = spark.conf.get("spark.databricks.workspaceUrl")
workspace_url = f"https://{instance_name}"
workspace_name = f"dbs-training-{environment}-01"
api_token = dbutils.secrets.get(scope = f"kv-training-01", key = f"dbs-training-{environment}-01-pat-token")
print("Databricks Instance URL:", workspace_url,"Databricks Instance Name:",workspace_name )

This notebook will extract all job configuration in central table using databricks api so we can see all job details in dashboard and we can easily take action. 
     Github confoguration 
     Cluster configration
     Execution schedule 

In [0]:

import requests
import pandas as pd
from datetime import datetime,timedelta

def fetch_job_information(api_token, workspace_url):
    headers = {
        "Authorization": f"Bearer {api_token}",
        "Content-Type": "application/json"
    }
    # Endpoint to list all jobs
    jobs_endpoint = f"{workspace_url}/api/2.0/jobs/list"
    
    # Sending GET request to fetch job information
    response = requests.get(jobs_endpoint, headers=headers)
    response_json = response.json()
    
    # Extracting relevant job information
    job_data = []
    for job in response_json["jobs"]:
        #print(job)
        job_id = job["job_id"]
        job_name = job["settings"]["name"]
        try:
            creator_name = job["creator_user_name"]
        except KeyError:
            creator_name = "Unknown"
        cluster_node_type_id = job["settings"].get("new_cluster", {}).get("node_type_id", "Unknown")
        cluster_spark_version = job["settings"].get("new_cluster", {}).get("spark_version", "Unknown")
        cluster_runtime_engine = job["settings"].get("new_cluster", {}).get("runtime_engine", "Unknown")
        cluster_num_workers = job["settings"].get("new_cluster", {}).get("num_workers", "Unknown")
        data_security_mode = job["settings"].get("new_cluster", {}).get("data_security_mode", "Unknown")
        job_created_time = datetime.fromtimestamp(job["created_time"]/ 1000)
        cron_expression = job["settings"].get("schedule", {}).get("quartz_cron_expression", "Unknown") 
        cron_timezone = job["settings"].get("schedule", {}).get("timezone_id", "Unknown")
        pause_status = job["settings"].get("schedule", {}).get("pause_status", "Unknown")
       
        # Appending job information to the list
        job_data.append((job_id, job_name,creator_name,cluster_node_type_id,cluster_spark_version,cluster_runtime_engine,cluster_num_workers,data_security_mode,job_created_time,cron_expression,cron_timezone,pause_status))
        
    # Creating a DataFrame from the extracted job information
    columns = ["job_id", "job_name","creator_name","cluster_node_type_id","cluster_spark_version","cluster_runtime_engine","cluster_num_workers","data_security_mode","job_created_time","cron_expression","cron_timezone","pause_status"]
    job_df = pd.DataFrame(job_data, columns=columns)
    
    return job_df

# Set your Databricks API token and workspace URL
api_token = dbutils.secrets.get(scope = f"kv-training-01", key = f"dbs-training-{environment}-01-pat-token")

# Fetch job information and store it in a DataFrame
job_information_df = fetch_job_information(api_token, workspace_url)

# New columns
workspace_name = workspace_name
workspace_url = workspace_url

# Inserting new columns
job_information_df.insert(2, 'workspace_name', workspace_name)
job_information_df.insert(3, 'workspace_url', workspace_url)

sparkdf = spark.createDataFrame(job_information_df)
sparkdf.createOrReplaceTempView("v_job_information")

Create config_dev/prod catalog for storing all configration details informations

In [0]:
InsertQuery = f"MERGE INTO config_{environment}.config.job_metadata_details AS target " \
              "USING v_job_information AS source " \
              "ON target.job_id = source.job_id " \
              "WHEN MATCHED THEN " \
              "  UPDATE SET target.job_name = source.job_name, " \
              "             target.cluster_node_type_id = source.cluster_node_type_id, " \
              "             target.cluster_spark_version = source.cluster_spark_version, " \
              "             target.cluster_runtime_engine = source.cluster_runtime_engine, " \
              "             target.cluster_num_workers = source.cluster_num_workers, " \
              "             target.data_security_mode = source.data_security_mode, " \
              "             target.cron_expression = source.cron_expression, " \
              "             target.cron_timezone = source.cron_timezone, " \
              "             target.pause_status = source.pause_status " \
              "WHEN NOT MATCHED THEN " \
              "  INSERT ( " \
              "          job_id, " \
              "          job_name, " \
              "          workspace_name , " \
              "          workspace_url , " \
              "          creator_name , " \
              "          cluster_node_type_id , " \
              "          cluster_spark_version , " \
              "          cluster_runtime_engine , " \
              "          cluster_num_workers , " \
              "          data_security_mode , " \
              "          job_created_time , " \
              "          cron_expression , " \
              "          cron_timezone , " \
              "          pause_status " \
              "        ) " \
              "  VALUES ( " \
              "            source.job_id, " \
              "            source.job_name, " \
              "            source.workspace_name , " \
              "            source.workspace_url , " \
              "            source.creator_name , " \
              "            source.cluster_node_type_id , " \
              "            source.cluster_spark_version , " \
              "            source.cluster_runtime_engine , " \
              "            source.cluster_num_workers , " \
              "            source.data_security_mode , " \
              "            source.job_created_time , " \
              "            source.cron_expression , " \
              "            source.cron_timezone , " \
              "            source.pause_status " \
              "          )"

Insert Job information metadata in job_metadata_details table 

In [0]:
Query = spark.sql(InsertQuery)

In [0]:
print("Total Inserted Records",Query.select('num_inserted_rows').collect())
print("Total update Records",Query.select('num_updated_rows').collect())
 

In [0]:
print("Succeeded")